# EXTRACTING FRAMES FROM A VIDEO FILE
---

This notebook lets you extract the frames of a given video file at the desired framerate and save them with numbered filenames as `.jpg` files in a new directory, keeping their native resolution.

First, let’s import the required librairies (openCV, numpy, os, math, datetime…)
> *(If you’re executing this notebook locally, you should install those with pip)*

In [1]:
import cv2
import numpy as np
import os
import math
#from datetime import timedelta
import datetime

Then, specify the path to the video file you’d like to extract the frames from.  

In [2]:
# enter file path to video (remote or local)
print("> Enter path to file")
video_file = str(input())
#ie. "http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/ForBiggerBlazes.mp4"

# get the video frame rate
cap = cv2.VideoCapture(video_file)
fps = cap.get(cv2.CAP_PROP_FPS)
frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# get the video duration (by dividing the total amount of frames by the video frame rate)
clip_duration = str(datetime.timedelta(seconds=int(frames/fps)))

if fps!=0:
  print("> This video contains", frames,"frames, sampled at", fps,"fps, for a total length of", clip_duration)
else:
  print("> No valid video found at this path")

> Enter path to file
http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/ForBiggerBlazes.mp4
> This video contains 360 frames, sampled at 23.976023976023978 fps, for a total length of 0:00:15


In [9]:
print("> Enter the frame rate at which you'd like to extract")
SAVING_FRAMES_PER_SECOND = float(input())
if SAVING_FRAMES_PER_SECOND<=fps:
  saving_frames_per_second = SAVING_FRAMES_PER_SECOND
  print("> At", SAVING_FRAMES_PER_SECOND, "FPS,", int(SAVING_FRAMES_PER_SECOND * (frames/fps)), "frames will be extracted in total.")
else:
  saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
  print("> /!\ The frame rate cannot be higher than the video's current rate. FPS set at", saving_frames_per_second)
# ie. 10

> Enter the frame rate at which you'd like to extract
12
> At 12.0 FPS, 180 frames will be extracted in total.


test

In [10]:
def get_saving_frames_durations(cap, saving_fps):
    """A function that returns the list of durations where to save the frames"""
    s = []
    # get the clip duration by dividing number of frames by the number of frames per second
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    # use np.arange() to make floating-point steps
    print("current frames in video:", math.floor(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
    print("duration of the video in seconds:", clip_duration)
    print("extracting frames:", saving_fps, "FPS")
    frame_count_prediction = saving_fps * clip_duration
    print("total frames expected:", math.floor(frame_count_prediction-1))
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s

test2

In [12]:
def main(video_file):
    filename, _ = os.path.splitext(video_file)
    filename += "-frames"
    # if there isn’t already one, create a folder named after filename + "-frames"
    if not os.path.isdir(filename):
        os.mkdir(filename)
    # read the video file
    cap = cv2.VideoCapture(video_file)
    # get the FPS of the video
    fps = cap.get(cv2.CAP_PROP_FPS)
    print("source video fps:", fps, "FPS")
    # if the SAVING_FRAMES_PER_SECOND is above video FPS, then set it to FPS (as maximum)
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    # get the list of duration spots to save
    saving_frames_durations = get_saving_frames_durations(cap, saving_frames_per_second)
    # start the loop
    count = 0
    index = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            # break out of the loop if there are no frames to read
            break
        # get the duration by dividing the frame count by the FPS
        frame_duration = count / fps
        try:
            # get the earliest duration to save
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # the list is empty, all duration frames were saved
            break

        if frame_duration >= closest_duration:
            # if closest duration is less than or equals the frame duration,
            # then save the frame
            #frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
#            cv2.imwrite(os.path.join(filename, str(video_file) + "-" + str(index) + ".jpg"), frame)
            cv2.imwrite(os.path.join(filename, str(video_file.rstrip('.mp4')) + "-" + str(index) + ".jpg"), frame)

            index += 1

            # drop the duration spot from the list, since this duration spot is already saved
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # increment the frame count
        count += 1

In [13]:
if __name__ == "__main__":
    main(video_file)

FileNotFoundError: ignored